Installing libraries

In [ ]:
!pip install streamlit -q # fro streamlit deployment.



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 36.5 MB/s eta 0:00:00


In [ ]:
!pip install -Uqq  git+https://github.com/huggingface/peft.git # for the model trained model deployment.
!pip install -Uqq transformers datasets accelerate bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install transformers==4.29.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.1
    Uninstalling transformers-4.33.1:
      Successfully uninstalled transformers-4.33.1


### Streamlit App

In [ ]:
#for the cpu code.
#device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#generated = generated.to(device)
#model.to(device)
#model.eval();
%%writefile app.py
import streamlit as st

st.title("HEALTHQUERY")


model = '3B' # Pick your poison'


if model == '7B':
    model_name = ("togethercomputer/RedPajama-INCITE-Base-7B-v0.1","togethercomputer/RedPajama-INCITE-Base-7B-v0.1")
    run_name = 'redpj7B-lora-int8-alpaca'
    dataset = 'johnrobinsn/alpaca-cleaned'
    peft_name = 'redpj7B-lora-int8-alpaca'
    output_dir = 'redpj7B-lora-int8-alpaca-results'
else: #3B
    model_name = ("togethercomputer/RedPajama-INCITE-Base-3B-v1","togethercomputer/RedPajama-INCITE-Base-3B-v1")
    run_name = 'redpj3B-lora-int8-alpaca'
    dataset = 'johnrobinsn/alpaca-cleaned'
    peft_name = 'redpj3B-lora-int8-alpaca'
    output_dir = 'redpj3B-lora-int8-alpaca-results'

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# load base LLM model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name[0],
    load_in_8bit=True,

    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name[1])
tokenizer.pad_token_id = 0
tokenizer.add_special_tokens({'eos_token':'<eos>'})

model.eval()

def generate_prompt(data_point):
  return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:"""


input_text = st.text_input("Please Provide your text:")

if len(input_text)>0:
  def generate(instruction,input=None,maxTokens=150):
      prompt = generate_prompt({'instruction':instruction,'input':input})
      input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
      outputs = model.generate(input_ids=input_ids, max_new_tokens=maxTokens,
                              do_sample=True, top_p=0.9,pad_token_id=tokenizer.eos_token_id,
                              forced_eos_token_id=tokenizer.eos_token_id,remove_invalid_values=True)
      outputs = outputs[0].tolist()
      # Stop decoding when hitting the EOS token
      if tokenizer.eos_token_id in outputs:
          eos_index = outputs.index(tokenizer.eos_token_id)
          decoded = tokenizer.decode(outputs[:eos_index])
          # Don't show the prompt template
          sentinel = "### Response:"
          sentinelLoc = decoded.find(sentinel)
          if sentinelLoc >= 0:
              st.write(decoded[sentinelLoc+len(sentinel):])
          else:
              print('Warning: Expected prompt template to be emitted.  Ignoring output.')
      else:
          print('Warning: no <eos> detected ignoring output')

  # peft_model_id = f'johnrobinsn/{peft_name}' # By default use my pretrained adapter weights
  peft_model_id = '/content/drive/MyDrive/All models/llama_redit'# Uncomment to use locally saved adapter weights if you trained above

  # Load the LoRA model
  model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
  model.eval()
  torch.manual_seed(42)
  title = input_text
  st.write(generate(title,maxTokens=150))

else:
  st.write('Welcome to HEALTHQUERY')


# Display output



Writing app.py


In [ ]:
!pip install pyngrok==5.2.1


from pyngrok import ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19772 sha256=6d24838e4202f82d736715eb7e34194fbac1e84ec9f9223009f4922425ae1ddc
  Stored in directory: /root/.cache/pip/wheels/6c/e1/46/8d60711cb43fb2e055fb69bb9964f91c9a5046f7924d2996ac
Successfully built pyngrok


In [ ]:
ngrok.set_auth_token("2LoeoedCqusM7VxjDHJfm1b9TJg_5wRMcFESTSHY6u26tUoMj") #ngrok.com




In [ ]:
!killall ngrok

In [ ]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://2b0c-34-168-44-182.ngrok-free.app" -> "http://localhost:80"
